In [16]:
import plotly.graph_objects as go
import pandas as pd

import matplotlib.pyplot as plt
import networkx as nx

# reading input log file
# import ruamel.yaml
import yaml
import re
import ast

import math
import sys
import traceback
from csv import excel
from collections import Counter
from scipy import stats

# test_name = "1p9f9s_run"
# test_name = "seq6f3s"
test_name = "seq9f9s"

stat_path=f"../example_stat/{test_name}"
iamge_path=f"{stat_path}/images"


In [17]:
import os
import re

def search_files_with_name(directory, pattern=""):
    file_list = []
    for root, dirs, files in os.walk(directory):
        print(root, dirs, files)
        for file in files:
            if re.search(pattern, file):
                file_list.append(os.path.join(root, file))
                #print(os.path.join(root, file))
    return file_list

vfd_files = search_files_with_name(stat_path, "vfd")
# vfd_files = vfd_files[0:1]
print(vfd_files)

../example_stat/seq9f9s [] ['44025_vfd-data-stat-dl.yaml', 'seq9f9s-task_to_file.yaml', '44025_vol-data-stat-dl.yaml', 'RUN_TRACKING-dl.log']
['../example_stat/seq9f9s/44025_vfd-data-stat-dl.yaml']


In [18]:

def load_vfd_yaml(vfd_files):
    # loag into {file_name:yaml_data} format
    ret_dict = {}
    tmp_dict = {}
    for f in vfd_files:
        with open(f, "r") as stream:
            print(f"loading {f}")
            try:
                tmp_dict = yaml.safe_load(stream)
                ret_dict[f] = tmp_dict
                # print(tmp_dict)
            except yaml.YAMLError as exc:
                print(exc)
                print("Error loading yaml file")
                exit(1)
    return ret_dict

vfd_file_dict = load_vfd_yaml(vfd_files)
print(vfd_file_dict.keys())


loading ../example_stat/seq9f9s/44025_vfd-data-stat-dl.yaml
dict_keys(['../example_stat/seq9f9s/44025_vfd-data-stat-dl.yaml'])


In [19]:
def extract_pid_from_file_path(file_path):
    # Define the regular expression pattern to find numbers in the file path
    pattern = r'\d+'
    # Use the re.findall function to find all occurrences of the pattern in the file path
    numbers = re.findall(pattern, file_path)
    # Since there might be multiple numbers in the file path, you can extract the last one
    if numbers:
        desired_number = int(numbers[-1])
        print("task_pid:", desired_number)
    else:
        print("No task_pid found in the file path.")
    return desired_number

def get_op_blob_range(h5fd_stat):
    # Format of io_stat
    '''
    read_bytes: 0
        read_cnt: 0
        read_ranges: {1:[0, 0]}
        write_bytes: 7746756
        write_cnt: 52
        write_ranges:{2:[0, 0]}
    '''
    read_ranges = h5fd_stat['read_ranges']
    write_ranges = h5fd_stat['write_ranges']
    
    if read_ranges:
        start_op_idx = min(read_ranges.keys())
        end_op_idx = max(read_ranges.keys())
        start_blob = min(elem[0] for elem in list(read_ranges.values())) #min(read_ranges.values())
        end_blob = max(elem[1] for elem in list(read_ranges.values()))
    if write_ranges:
        start_op_idx = min(write_ranges.keys())
        end_op_idx = max(write_ranges.keys())
        start_blob = min(elem[0] for elem in list(write_ranges.values())) #min(write_ranges.values())
        end_blob = max(elem[1] for elem in list(write_ranges.values()))
    
    result = {
        'start_op_idx': start_op_idx,
        'end_op_idx': end_op_idx,
        'start_blob': start_blob,
        'end_blob': end_blob
    }
    
    return result

# TODO: need different get_min_max_op_idx
def get_min_max_op_idx(io_stat):
    # assumes we want the I/O operations and blobs for per file
    min_op_idx = -1
    max_op_idx = -1
    min_blob = -1
    max_blob = -1
    for h5fd_type, h5fd_stat in io_stat.items():
        res_stat = get_op_blob_range(h5fd_stat)
        
        if min_op_idx == -1:
            min_op_idx = res_stat['start_op_idx']
        elif res_stat['start_op_idx'] < min_op_idx:
            min_op_idx = res_stat['start_op_idx']
        
        if max_op_idx == -1:
            max_op_idx = res_stat['end_op_idx']
        elif res_stat['end_op_idx'] > max_op_idx:
            max_op_idx = res_stat['end_op_idx']
        
        if min_blob == -1:
            min_blob = res_stat['start_blob']
        elif res_stat['start_blob'] < min_blob:
            min_blob = res_stat['start_blob']
        
        if max_blob == -1:
            max_blob = res_stat['end_blob']
        elif res_stat['end_blob'] > max_blob:
            max_blob = res_stat['end_blob']
    
    result = {
        'min_op_idx': min_op_idx,
        'max_op_idx': max_op_idx,
        'min_blob': min_blob,
        'max_blob': max_blob
    }
    
    # print(f"h5fd_type: {h5fd_type}")
    # print(f"res_stat: {result}")
    
    return result


# Converting to Task-File dictionary
def schema_to_task_file_dic(vfd_file_dict):
    task_file_dict = {}
    # Format of task_file_dict
    '''
    run_idfeature:
        order: 0
        task_pid: 75190
        task_op_range: [0, 7208]
        total_op_range: [0, 7208]
        input:
            - file_name: "wrfout_rainrate_tb_zh_mh_2015-05-06_04:00:00.nc"
                file_op_range: [55, 71]
        output:
            - file_name: "cloudid_20150506_040000.nc"
                ile_op_range: [0, 800]
    '''
    for file_path, file_data in vfd_file_dict.items():

        task_pid = extract_pid_from_file_path(file_path)
        
        for li in file_data:
            # print(li.keys())
            if 'file' in list(li.keys())[0]:
                key = list(li.keys())[0]
                # print(li[key])
                
                if li[key]['file_type'] == 'na':
                    print("Invalid Entry. Skipping...")
                    continue

                task_name = li[key]['task_name']
                file_name = li[key]['file_name']

                if task_name not in task_file_dict.keys():
                    # 1. Initialize task_file_dict
                    task_file_dict[task_name] = {}
                    task_file_dict[task_name]['order'] = 0 # placeholder
                    task_file_dict[task_name]['task_pid'] = task_pid
                    task_file_dict[task_name]['io_cnt'] = 0 # initial value
                    # task_file_dict[task_name]['total_op_range'] = []
                    task_file_dict[task_name]['input'] = {}
                    task_file_dict[task_name]['output'] = {}
                    
                # 1. Update task op count
                read_cnt = li[key]['file_read_cnt']
                write_cnt = li[key]['file_write_cnt']
                task_file_dict[task_name]['io_cnt'] += (read_cnt + write_cnt)
                
                # 2. Get blob and operation info
                data_stat = li[key]['data']#['H5FD_MEM_DRAW']
                meta_stats = li[key]['metadata']
                data_stat.update(meta_stats)

                # try: 
                # except:
                #     print("Error:")
                #     print(f"task_name: {task_name}")
                #     print(f"file_name: {file_name}")
                io_stat = get_min_max_op_idx(data_stat)
                
                # 2. Add file to intput/output list
                file_type = li[key]['file_type']
                if file_type == 'input':
                    # task_file_dict[task_name]['input'].append(file_name)
                    task_file_dict[task_name]['input'][file_name] = io_stat
                elif file_type == 'output':
                    # task_file_dict[task_name]['output'].append(file_name)
                    task_file_dict[task_name]['output'][file_name] = io_stat
                # TODO: what if read_write file occur?
                

                # print(f"{file_name}: {io_stat}")
                # data_stat_range_list = list(data_stat['write_ranges'].values())
                
    return task_file_dict

def sort_task_in_order(task_file_dict, task_order_list):
    op_offset = 0
    for i, task_name in enumerate(task_order_list):
        # if task_name in task_file_dict.keys():
        task_file_dict[task_name]['order'] = i
        io_cnt = task_file_dict[task_name]['io_cnt']
        # task_file_dict[task_name]['total_op_range'] = f"[{op_offset},{io_cnt}]"
        task_file_dict[task_name]['total_op_range'] = {}
        
        task_file_dict[task_name]['total_op_range']['start'] = op_offset
        task_file_dict[task_name]['total_op_range']['end'] = op_offset + io_cnt

        op_offset += io_cnt
    return task_file_dict


In [20]:
# Save the task to input/output file mapping
def save_task_file_dict(task_file_dict, test_name):
    tf_file_path = stat_path=f"../example_stat/{test_name}/{test_name}-task_to_file.yaml"
    
    # if os.path.exists(tf_file_path):
    #     # Remove the file
    #     os.remove(tf_file_path)
    
    with open(tf_file_path, 'w') as yaml_file:
        for task_name,data in task_file_dict.items():
            input_files = list(data['input'].keys())
            output_files = list(data['output'].keys())
            item_dict = {
                task_name: {
                    'order': data['order'],
                    'task_pid': data['task_pid'], # TODO: not need?
                    'io_cnt': data['io_cnt'], # TODO: not need?
                    'input': input_files,
                    'output': output_files
                }
            }
            yaml.dump(item_dict, yaml_file)

# task_order_list = [ "run_idfeature", "run_tracksingle", "run_gettracks", "run_trackstats", "run_identifymcs", "run_matchpf", "run_mcsstats", "run_robustmcs", "run_mapfeature", "run_speed"]

task_file_dict = schema_to_task_file_dic(vfd_file_dict)
task_order_list = [ "run_idfeature", "run_tracksingle", "run_gettracks"]
ordered_task_file_dict = sort_task_in_order(task_file_dict, task_order_list)
save_task_file_dict(task_file_dict, test_name)

task_pid: 44025
Invalid Entry. Skipping...


In [21]:
# Convert dictionary to prefetcher format

def save_hermes_prefetch(task_file_dict, test_name):

    prefetch_file_path = f"../example_stat/{test_name}/apriori_{test_name}.yaml"

    with open(prefetch_file_path, 'w') as file:
        file.write("0:\n")
        # for task_name,data in task_file_dict.items():
        for data in task_file_dict.values():
            for input_file,file_stat in data['input'].items():
                op_range = f"[{file_stat['min_op_idx']},{file_stat['max_op_idx']}]"
                blob_range = f"[{file_stat['min_blob']},{file_stat['max_blob']}]"
                file.write(f"  - bucket: {input_file}\n")
                file.write(f"    prefetch:\n")
                file.write(f"    - op_count_range: {op_range}\n")
                file.write(f"      promote_blobs: {blob_range}\n")
            for output_file,file_stat in data['input'].items():
                op_range = f"[{file_stat['min_op_idx']},{file_stat['max_op_idx']}]"
                blob_range = f"[{file_stat['min_blob']},{file_stat['max_blob']}]"
                file.write(f"  - bucket: {output_file}\n")
                file.write(f"    prefetch:\n")
                file.write(f"    - op_count_range: {op_range}\n")
                file.write(f"      demote_blobs: {blob_range}\n")
            # print(input_file)
            # print(file_stat)

save_hermes_prefetch(task_file_dict, test_name)    
    